# `bw_timex` Teaching Exercise - Time-explicit LCA of an Electric Vehicle vs Petrol Vehicle.

In this exercise notebook you will compare the climate preformance of a battery electric vehicle (BEV) and an internal combustion engine (ICE) car, in a time explicit way, using in different decarbonisation scenarios. 

Optional challenge at the end: 

- using temporal distributions, model the material requirements of a fleet of electric cars/busses.


In [ ]:
# first we need to load the correct Brightway2 project with the required databases
import bw2data as bd

# project_name = "Timex_teaching"
project_name = "BAFU_report"

if project_name:# not in bd.projects:
    import bw2io as bi
    bi.backup.restore_project_directory(fp= '/Users/ajakobs/Documents/SCENE/LCA model/databases/base_projects/brightway2-project-BAFU_report-backup07-November-2024-12-21PM.tar.gz',
                                    overwrite_existing=True)

bd.projects.set_current(project_name)

In [ ]:
# check the databases present
print(bd.databases)

## Prospective background databases

We used `premise` to create two different sets of 'ecoinvent-databases' representing two different carbonisation pathways. They are the based on the REMIND-EU IAM for the SSP2 scenario in combination with Paris Agreement consistent policies (PkBud650) as an (very) optimistic scenario, and the 'national policies implemented' (NPi) as a business as usual (BAU) scenario:

- REMIND-EU - SSP2-NPi       : BAU scenario with ~3˚ warming by 2100
- REMIND-EU - SSP2-PkBudg650 : Optimistic scenario in line with Paris 2015 with 1.5˚-1.9˚ warming bby 2100

(See [the Premise documentation](https://premise.readthedocs.io/en/latest/introduction.html#default-iam-scenarios) for a brief overview of the IAM scenarios) 



In [3]:
# Let's get easy handles for the databases for ease of use:

db_2020 = bd.Database("ei_cutoff_3.10_image_SSP2-Base_2020 2024-11-07")
db_2030 = bd.Database("ei_cutoff_3.10_image_SSP2-Base_2030 2024-11-07")
db_2040 = bd.Database("ei_cutoff_3.10_image_SSP2-Base_2040 2024-11-07")
db_2050 = bd.Database("ei_cutoff_3.10_image_SSP2-Base_2050 2024-11-07")


In [4]:
# db_2020_NPi = bd.Database("ei_cutoff_3.11_remind-eu_SSP2-NPi_2020 2025-10-01")
# db_2030_NPi = bd.Database("ei_cutoff_3.11_remind-eu_SSP2-NPi_2030 2025-10-01")
# db_2040_NPi = bd.Database("ei_cutoff_3.11_remind-eu_SSP2-NPi_2040 2025-10-01")
# db_2050_NPi = bd.Database("ei_cutoff_3.11_remind-eu_SSP2-NPi_2050 2025-10-01")

## Case study setup

In this exercise we will go about setting up the forground system in a slightly different way. In our case driving a(n electric) car is already represented in ecoinvent. So all we would need to do is to add the temporal distributions to the exchanges. In the current implementation of `bw_timex`, all activities 
containing temporal information on the exchanges need to 'live' in the foreground system, so that their inputs can get linked to different background activities.

Let's start the definition of our functional unit (FU): 

- Driving a passenger car for 200,000km (including production, EoL treatment, maintenance, etc), over a period of 15 years.




### Modeling the production system

As a first step of modelling the production system let's create a new foreground database:


In [5]:
fg_db_name = "foreground"

if fg_db_name in bd.databases:
    del bd.databases[fg_db_name] # to make sure we create the foreground from scratch
foreground = bd.Database(fg_db_name)
foreground.register()

And let's define the FU relevant quantities: 

In [ ]:
LIFETIME_KM = 200000.  # lifetime of the vehicle in km
LIFETIME = 15  # lifetime of the vehicle in years
ANNUAL_KM = LIFETIME_KM / LIFETIME  # annual km

Now let's create two activities in the foreground that represent the functional units as defined above (production/driving/EoL treatment of a car for 200,000km over 15 years). We can call them LC_BEV and LC_ICEC for example (LC=life cyle)

Ecoinvent already contains activities that represent transport with a passenger car. So let's get the following two activities from the base database** for the BAU scenario and create a copy in our foreground database:

- Driving a gasoline car: "transport, passenger car, gasoline, Medium SUV, EURO-6"
- Driving a BEV: "transport, passenger car, gasoline, Medium SUV, EURO-6"

HINT: copy to a different database by first copying the activity and then changing the database.

** We consider that as our base year and db_2020 as the 'base' database

The Reference product of these activities should be 1 km traveled by the resepctive vehicle, but let's check the reference product and its unit:

Have a look at what the inputs into such a car is and identify the relevant exchanges:

These activities include both the car, which in itself contains both production and EoL treatment (check this!!), as well as the fuel/electricity, maintenance and wear etc. Now we want to model the production and EoL treatment of car and it's elements separately so let's make a copy of these activities in the foreground so that we can add temporal distributions to the production and EoL.

At the same time we want to split the driving and production/EoL so set the input of the car into the driving activity to 0 to avoid double counting.

Now it is time to add the relevant exchanges to the functional unit activities together with relevant temporal distributions. Be carefull to add the temporal distribution to poduction and EoL at the right exchnage, so that production happens before the use phase but EoL after the use phase. 

Tipp: for ease of use, or if you forgot how to define temporal distributions you can use the interactive_td_widget!

Tipp 2: Use the utility function add_temporal_distribution_to_exchange to add the temporal distribution to the EoL of the production/EoL activity of the cars. 


### Checking all went right

Now before calculating the time-explicit LCA, make sure that everything went right so far. As a check, the standard LCA score of the original transport activity with the corresponding amount should be equal the standard LCA of the modified life cycle activity created in the foreground!

## Time-explicit LCA with `bw_timex`

Now we can start using `bw_timex` to build the process timeline, build the time-explicit inventory and calculate the time explicit scores.

Start by creating a dictionary mapping the respective databases to the relevant timestamps. 
Then instantiate your timexLCA object, build the timeline, calculate the lci, and the LCIA for both options.


## Visualise Results and compare the different cars and scenarios

now find a good way to visualise the results. There are some simple plotting functions included such as plot_dynamic_inventory().

Run the other scenario to see the differences for both cars. 

## BONUS options:

- Use dynamic_lcia(use_disaggregated_lci=TRUE) to perform a contribution analysis on the background activities contributing to the impacts. (Note that this function can take a while), find a good way to visualize thi

- Adavanced: Use temporal distritbutions to modell a fleet (of changing composition). (hint a for a fleet some cars will live longer, some shorter. This can be modelled using a distribution for the usephase and the EoL)



